In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential
from tensorflow.python.client import device_lib

2023-12-07 12:24:39.593132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
image_size = (48, 48)
batch_size = 64

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "data/data/images/train",
    label_mode='categorical',
    validation_split=0.2,
    subset="both",
    shuffle=True,
    seed=666,
    image_size=image_size,
    batch_size=batch_size,
)

Found 28821 files belonging to 7 classes.
Using 23057 files for training.
Using 5764 files for validation.


In [27]:
def make_model(input_shape, num_classes):
    
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=1, padding="same")(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, strides=1, padding="same")(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dropout(0.2)(x)
    x = keras.layers.Flatten()(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    return keras.Model(inputs, outputs)

In [28]:
model = make_model(input_shape=image_size + (3,), num_classes=7)


In [29]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 rescaling_4 (Rescaling)     (None, 48, 48, 3)         0         
                                                                 
 conv2d_10 (Conv2D)          (None, 48, 48, 32)        896       
                                                                 
 activation_10 (Activation)  (None, 48, 48, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 48, 48, 64)        18496     
                                                                 
 activation_11 (Activation)  (None, 48, 48, 64)        0         
                                                                 
 global_average_pooling2d_4  (None, 64)                0   

In [30]:
epochs = 15

callbacks = [
    keras.callbacks.ModelCheckpoint("models/save_at_{epoch}.keras"),
    keras.callbacks.EarlyStopping(patience=5)
]
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/15


361/361 [==============================] - ETA: 0s - loss: 1.8222 - accuracy: 0.2425

In [21]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [25]:
base_model = ResNet50V2(include_top= False, input_shape=(48, 48, 3), weights='imagenet', classifier_activation= 'softmax')

94668760/94668760 [==============================] - 6s 0us/step


Epoch 1/15


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
     